In [4]:
import MySQLdb
from config import config
import pandas as pd
mysql_config = config.get('MYSQL_DB')
con= MySQLdb.connect(**mysql_config) #return Connection object
query = """SELECT * FROM northwind.customers;"""
cursor = con.cursor() #Connection.cursor() return Cursor object
try: 
    cursor.execute(query)
    # cursor.fetchone() # return tuple
    # cursor.fetchmany(5) # return tuple of tuple
    data = cursor.fetchall() # return tuple of tuple
    con.commit()
except Exception as e:
    print('Error: ', e)
    con.rollback()
data = pd.DataFrame(data)
print(data[:5])

      0                                   1                   2   \
0  ALFKI                 Alfreds Futterkiste        Maria Anders   
1  ANATR  Ana Trujillo Emparedados y helados        Ana Trujillo   
2  ANTON              Antonio Moreno Taquera      Antonio Moreno   
3  AROUT                     Around the Horn        Thomas Hardy   
4  BERGS                   Berglunds snabbkp  Christina Berglund   

                     3                             4           5     6   \
0  Sales Representative                 Obere Str. 57      Berlin  None   
1                 Owner  Avda. de la Constitucin 2222  Mxico D.F.  None   
2                 Owner               Mataderos  2312  Mxico D.F.  None   
3  Sales Representative               120 Hanover Sq.      London  None   
4   Order Administrator                Berguvsvgen  8        Lule  None   

         7        8               9               10  
0     12209  Germany     030-0074321     030-0076545  
1     05021   Mexico    (5) 55

In [ ]:
#example of cursor.execute(query, params)
c=db.cursor()
max_price=5
c.execute("""SELECT spam, eggs, sausage FROM breakfast
          WHERE price < %s""", (max_price,))

**Note about mysqlclient package:**
- mysqlclient package is one of the best mysql client package.
- it allow client to make query to the mysql database
- we have important class like: Connection and Cursor
- most of the action will be done by Cursor object.
- Connection methods: commit() to make change to database, rollback() to rollback execution, cursor() to make Cursor instance
- Cursor methods: execute(query, params), executemany